# Fitting code using Gaussian modulation patterns on simulated measurement data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
import re
import os.path
from os import path
from definitions import *
from instrument import *
from util import *

In [2]:
def compute_P(I_up,I_down, dI_up, dI_down):
    P = (I_up - I_down) / (I_up + I_down)
    dP = 2 * np.sqrt((I_down * dI_up)**2 + (I_up * dI_down)**2) / (I_up + I_down) ** 2 
    return P, dP

def compute_I_diff(I_up,I_down, dI_up, dI_down):
    I_diff = (I_up - I_down)
    dI_diff = np.sqrt(dI_up ** 2 + dI_down ** 2)
    return I_diff, dI_diff

def compute_rms(x, dx):
    rms = np.sqrt(np.mean(np.square(x)))
    squared_errors = np.square(dx)
    d_rms = np.sqrt(np.mean(squared_errors) / len(x))
    return rms, d_rms 

def get_measurement_point(i, folder='data'):
    d1 = np.genfromtxt(f'{folder}/up/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    d2 = np.genfromtxt(f'{folder}/down/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    d3 = np.genfromtxt(f'{folder}/empty_up/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    d4 = np.genfromtxt(f'{folder}/empty_down/{i}/det.dat', delimiter=' ', usecols=(0,1,2,3), unpack=True)
    up = DetectorReading(*d1)
    down = DetectorReading(*d2)
    empty_up = DetectorReading(*d3)
    empty_down = DetectorReading(*d4)
    parameters = extract_parameters(f'{folder}/up/{i}/det.dat')
    mp = MeasurementPoint(up, down, empty_up, empty_down, parameters)
    return mp
    

class DetectorReading:
    def __init__(self, y, I, I_err, N):
        self.y = y * 10
        self.I = I
        self.dI = I_err
        self.N = N

class MeasurementPoint:
    def __init__(self, up, down, empty_up, empty_down, parameters):
        self.up = up
        self.down = down
        self.empty_up = empty_up
        self.empty_down = empty_down
        self.parameters = parameters
    
    def y(self):
        return self.up.y
    
    # Computes just I_up - I_down
    def compute_I_diffs(self):
        I_diff_s, dI_diff_s = compute_I_diff(self.up.I, self.down.I, self.up.dI, self.down.dI)
        I_diff_b, dI_diff_b = compute_I_diff(self.empty_up.I, self.empty_down.I, self.empty_up.dI, self.empty_down.dI)
        return self.y(), I_diff_b, dI_diff_b, I_diff_s, dI_diff_s

    def compute_P(self):
        P_s, dP_s = compute_P(self.up.I, self.down.I, self.up.dI, self.down.dI)
        P_b, dP_b = compute_P(self.empty_up.I, self.empty_down.I, self.empty_up.dI, self.empty_down.dI)
        return self.y(), P_b, dP_b, P_s, dP_s
    
    def compute_rms(self, y_range = None):
        y, P_b, dP_b, P_s, dP_s = self.compute_P()
        # y, P_b, dP_b, P_s, dP_s = self.compute_I_diffs()
        if y_range == None:
            rms_b, d_rms_b = compute_rms(P_b, dP_b)
            rms_s, d_rms_s = compute_rms(P_s, dP_s)
        else:
            # range is assumed to be of the form (y_min, y_max)
            indices = indices_within_range(y, *y_range)
            rms_b, d_rms_b = compute_rms(P_b[indices], dP_b[indices])
            rms_s, d_rms_s = compute_rms(P_s[indices], dP_s[indices])
        return rms_b, d_rms_b, rms_s, d_rms_s

    def rms_ratio(self, y_range = None):
        rms_b, d_rms_b, rms_s, d_rms_s = self.compute_rms(y_range=y_range)
        R = rms_s / rms_b
        dR = np.sqrt((d_rms_s / rms_b)**2 + (rms_s * d_rms_b / rms_b**2)**2)
        return R, dR
    
    def plot_intensities(self):
        plt.plot(self.y(), self.empty_up.I, label='Up')
        plt.plot(self.y(), self.empty_down.I, label = 'Down')
        plt.title(r'Base $I_{up}, I_{down}$')
        plt.grid()
        plt.legend()
        plt.show()    
        plt.plot(self.y(), self.up.I, label='Up')
        plt.plot(self.y(), self.down.I, label = 'Down')
        plt.title(r'Sample $I_{up}, I_{down}$')
        plt.grid()
        plt.legend()
        plt.show()


In [ ]:
def fit_gauss(y, I, dI, y_range, By, lambda_0, theta_0, wl_sigma):   
    indices = indices_within_range(y * 1e-3, *y_range)
    def I_fit(y, A_0, I_0):
        return I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=wl_sigma) * A_0 + I_0
    # Todo: use dI for better estimate?
    popt, pcov = curve_fit(I_fit, y[indices], I[indices], absolute_sigma=True,sigma = dI[indices])
    # print(popt, pcov)
    A_0 = popt[0]
    dA_0 = np.sqrt(pcov[0][0])
    I_0 = popt[1]
    dI_0 = np.sqrt(pcov[1][1])
    # print(I_0, dI_0)
    return A_0, dA_0, I_0, dI_0

# Gaussian envelope and RMS fitting 

In [ ]:
y_lim = 3e-3 # m
y_range=(-y_lim,y_lim) # m

intrs = load_instruments('simulations_new.csv')
for instr in intrs[3:-6]:
    for fit_method in ['gauss', 'rms']:
        print(f"FITTING USING {fit_method.upper()}")
        for radius in [500,3000, 20000]:
            N = 30
            B = np.zeros(N)
            delta = np.zeros(N)
            P = np.zeros(N)
            dP = np.zeros(N)
            # P_rms = np.zeros(N)
            # dP_rms = np.zeros(N)
            for i in range(N):
                measurement = get_measurement_point(i, f'working-data/data_{instr.id}_{radius}')
                y, P_b, dP_b, P_s, dP_s = measurement.compute_P()
                By = float(measurement.parameters['By'])
                lambda_0 = float(measurement.parameters['L0']) * 1e-10
                lambda_sigma = float(measurement.parameters['DL']) * 1e-10
                theta_0 = float(measurement.parameters['theta0'])
                try: 
                    instr.L_s = float(measurement.parameters['L_s'])
                except:
                    pass
                B[i] = By
                delta[i] = compute_z(B[i],instr.theta_0,instr.L0,instr.L_s)

                if fit_method == 'gauss':
                    # Gaussian fit
                    A_0_b, dA_0_b, I_0_b, dI_0_b = fit_gauss(y, P_b, dP_b, y_range, By, lambda_0, theta_0, lambda_sigma)
                    A_0_s, dA_0_s, I_0_s, dI_0_s = fit_gauss(y, P_s, dP_s, y_range, By, lambda_0, theta_0, lambda_sigma)
                    R = A_0_s / A_0_b
                    dR = np.sqrt((dA_0_s / A_0_b)**2 + (A_0_s * dA_0_b / A_0_b**2)**2)
                    P[i] = R
                    dP[i] = dR
                elif fit_method == 'rms':
                    P[i], dP[i] = measurement.rms_ratio(y_range)
            min_delta, max_delta = np.min(delta), np.max(delta)
            t = float(measurement.parameters['t'])
            R = float(measurement.parameters['R']) * 1e-10
            delta_range = np.linspace(0, R * 3, 1000)
            wavelength = float(measurement.parameters['L0']) * 1e-10
            P_an = P_analytical(delta_range, R, t, wavelength)
            # plt.title(rf"$P(\delta)$ for instrument {name}")
            # plt.plot(delta * 1e9,P, '.',label=rf'$R = {round(R*1e9)}$nm, fit')
            plt.errorbar(delta * 1e9,P, ms=3,fmt='o', linestyle=' ', yerr=dP,label=rf'$R = {round(R*1e9)}$nm, fit')
            plt.plot(delta_range * 1e9, P_an, lw = 1.5, label=rf'$R = {round(R*1e9)}$nm, analytical')
        name = instr.name
        plt.xscale('log')
        plt.xlim((10, R * 3 * 1e9))
        
        plt.xlabel(r'$\delta$(nm)')
        plt.ylabel(r'$P(\delta)$')
        plt.legend()
        plt.grid()
        plt.savefig(f"docs/simulation-plot-{fit_method}-{name.replace(' ','-')}.eps",bbox_inches="tight", pad_inches=0, format='eps')
        plt.show()

# Raw data plots and demonstration
It is important to also show some of the data you perform a fitting routine on etc. to illustrate.

In [ ]:
y_lim = 3e-3 # m
y_range=(-y_lim,y_lim) # m

intrs = load_instruments('simulations_new.csv')
instr = intrs[7]

radius = 3000
i = 29
measurement = get_measurement_point(i, f'working-data/data_{instr.id}_{radius}')
y, P_b, dP_b, P_s, dP_s = measurement.compute_P()
By = float(measurement.parameters['By'])
lambda_0 = float(measurement.parameters['L0']) * 1e-10
lambda_sigma = float(measurement.parameters['DL']) * 1e-10
theta_0 = float(measurement.parameters['theta0'])
try: 
    instr.L_s = float(measurement.parameters['L_s'])
except:
    pass
delta = compute_z(By,instr.theta_0,instr.L0,instr.L_s)
t = float(measurement.parameters['t'])
R = float(measurement.parameters['R']) * 1e-10

# Gauss
A_0_b, dA_0_b, I_0_b, dI_0_b = fit_gauss(y, P_b, dP_b, y_range, By, lambda_0, theta_0, lambda_sigma)
A_0_s, dA_0_s, I_0_s, dI_0_s = fit_gauss(y, P_s, dP_s, y_range, By, lambda_0, theta_0, lambda_sigma)
R = A_0_s / A_0_b
dR = np.sqrt((dA_0_s / A_0_b)**2 + (A_0_s * dA_0_b / A_0_b**2)**2)
P_gauss = R
dP_gauss = dR
# RMS
P_rms, dP_rms = measurement.rms_ratio(y_range)
wavelength = float(measurement.parameters['L0']) * 1e-10

P_an_val = P_analytical(delta, radius * 1e-10, t, wavelength)
print(P_rms, dP_rms, P_an_val)
print(P_gauss, dP_gauss, P_an_val)


In [ ]:
I_empty_up = measurement.empty_up.I
I_empty_down = measurement.empty_down.I
I_up = measurement.up.I
I_down = measurement.down.I
dI_empty_up = measurement.empty_up.I
dI_empty_down = measurement.empty_down.I
dI_up = measurement.up.I
I_down = measurement.down.I
y = measurement.y()

def plot_intensity(I, label=''):
    plt.plot(y, I ,'.',ms=3)
    plt.xlabel(r'$y$ [mm]')
    plt.ylabel(r'$I$ [a.u.]')
    plt.grid()
    plt.savefig(f"docs/simulation-raw-intensity-{label}.eps",bbox_inches="tight", pad_inches=0, format='eps')
    plt.show() 


for (label,I) in [('b-up',I_empty_up), ('b-down', I_empty_down), ('s-up', I_up), ('s-down',I_down)]:
    plot_intensity(I,label)

In [ ]:
plt.plot(y, I_empty_up ,'.',ms=3, label=r'$I_{b,up}$')
plt.plot(y, I_up ,'.',ms=3, label=r'$I_{s,up}$')
plt.xlabel(r'$y$ [mm]')
plt.ylabel(r'$I$ [a.u.]')
plt.grid()
plt.legend()
plt.savefig(f"docs/simulation-raw-intensity-up.eps",bbox_inches="tight", pad_inches=0, format='eps')
plt.show() 
plt.plot(y, I_empty_down ,'.',ms=3, label=r'$I_{b,down}$')
plt.plot(y, I_down ,'.',ms=3, label=r'$I_{s,down}$')
plt.xlabel(r'$y$ [mm]')
plt.ylabel(r'$I$ [a.u.]')
plt.grid()
plt.legend()
plt.savefig(f"docs/simulation-raw-intensity-down.eps",bbox_inches="tight", pad_inches=0, format='eps')
plt.show()

In [ ]:
def I_fit(y, A_0, I_0):
    return I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=lambda_sigma) * A_0 + I_0
y, P_b, dP_b, P_s, dP_s = measurement.compute_P()

plt.plot(y, I_empty_up + I_empty_down,'.',ms=3, label=r'$I_{b,up} + I_{b,down}$')
plt.plot(y, I_up + I_down ,'.',ms=3, label=r'$I_{s,up} + I_{s,down}$')
plt.plot(y, I_empty_up - I_empty_down,'.',ms=3, label=r'$I_{b,up} - I_{b,down}$')
plt.plot(y, I_up - I_down ,'.',ms=3, label=r'$I_{s,up} - I_{s,down}$')
plt.xlabel(r'$y$ [mm]')
plt.ylabel(r'$I$ [a.u.]')
plt.grid()
plt.legend()
plt.savefig(f"docs/simulation-raw-intensity-differential.eps",bbox_inches="tight", pad_inches=0, format='eps')
plt.show() 
plt.plot(y, P_b ,'.',ms=3, label=r'$Pol_b$')
plt.plot(y, P_s ,'.',ms=3, label=r'$Pol_s$')

fit_line_s = I_fit(y, A_0_s, I_0_s)
fit_line_b = I_fit(y, A_0_b, I_0_b)
plt.plot(y, fit_line_b, lw = 1.5, label=r'$Pol_{fit,b}$')
plt.plot(y, fit_line_s, lw = 1.5, label=r'$Pol_{fit,s}$')
plt.axvline(y_lim * 1e3, linestyle='--', color='red')
plt.axvline(-y_lim * 1e3, linestyle='--', color='red')
plt.xlabel(r'$y$ [mm]')
plt.ylabel(r'$Pol$ [a.u.]')
plt.grid()
plt.legend()
plt.savefig(f"docs/simulation-raw-intensity-pol.eps",bbox_inches="tight", pad_inches=0, format='eps')
plt.show()